# Analyze eBPF Programs

This notebook compiles, analyzes, and generates the report for eBPF programs. 

The raw report is saved in the `output/report` directory.

In [1]:
%reload_ext autoreload
%autoreload 2

The following cell compiles the BCC tools. 

In [2]:
from utils import BCC_TOOLS_PATH

!make -C {BCC_TOOLS_PATH} -j $(nproc)

make: Entering directory '/users/szhong/DepSurf/software/bcc/libbpf-tools'
  CC       profile.o
  BPF      readahead.bpf.o
  BINARY   profile
  GEN-SKEL readahead.skel.h
  CC       readahead.o
  BINARY   readahead
make: Leaving directory '/users/szhong/DepSurf/software/bcc/libbpf-tools'


The following cell compiles the eBPF program for Tracee.

In [3]:
from utils import TRACEE_PATH

!make -C {TRACEE_PATH} bpf

make: Entering directory '/users/szhong/DepSurf/software/tracee'
make: Nothing to be done for 'bpf'.
make: Leaving directory '/users/szhong/DepSurf/software/tracee'


The following cell analyzes and generates the report for eBPF programs.

In [4]:
from depsurf import BPFProgram, VersionGroup, gen_report
from utils import TRACEE_PATH, BCC_OBJ_PATH, OUTPUT_PATH, save_pkl

REPORT_PATH = OUTPUT_PATH / "report"
REPORT_PATH.mkdir(parents=True, exist_ok=True)
GROUPS = [VersionGroup.REGULAR, VersionGroup.ARCH]
PROG_PATHS = list(BCC_OBJ_PATH.glob("*.bpf.o")) + [TRACEE_PATH / "dist" / "tracee.bpf.o"]

results = {}
for path in PROG_PATHS:
    bpf = BPFProgram(path)
    result_path = REPORT_PATH / f"{bpf.name}.log"
    results[bpf.name] = gen_report(bpf.deps, GROUPS, path=result_path)


save_pkl(results, "bcc")

[         dump.py:162] INFO: Normalizing types from /users/szhong/DepSurf/software/bcc/libbpf-tools/.output/tcpstates.bpf.min.btf.json
[        types.py:42 ] INFO: Loading types from /users/szhong/DepSurf/data/dataset/types_struct/4.4.0-21-generic-amd64.jsonl
[        types.py:42 ] INFO: Loading types from /users/szhong/DepSurf/data/dataset/types_struct/4.8.0-22-generic-amd64.jsonl
[        types.py:42 ] INFO: Loading types from /users/szhong/DepSurf/data/dataset/types_struct/4.10.0-19-generic-amd64.jsonl
[        types.py:42 ] INFO: Loading types from /users/szhong/DepSurf/data/dataset/types_struct/4.13.0-16-generic-amd64.jsonl
[        types.py:42 ] INFO: Loading types from /users/szhong/DepSurf/data/dataset/types_struct/4.15.0-20-generic-amd64.jsonl
[        types.py:42 ] INFO: Loading types from /users/szhong/DepSurf/data/dataset/types_struct/4.18.0-10-generic-amd64.jsonl
[        types.py:42 ] INFO: Loading types from /users/szhong/DepSurf/data/dataset/types_struct/5.0.0-13-generi